In [1]:
import pickle 
import json
import pandas as pd
import numpy as np

In [2]:
with open('RPs.pkl' , 'rb') as fp:
    d = pickle.load(fp)

In [3]:
with open('meta.json' , 'r') as fp:
    meta = json.load(fp)

In [4]:
'''
d has the following structure -> {location : RSSI value}


location - > room number.location in that room.timestamp.random
RSSI value -> a vector of length len(AP_LIST)
'''

'\nd has the following structure -> {location : RSSI value}\n\n\nlocation - > room number.location in that room.timestamp.random\nRSSI value -> a vector of length len(AP_LIST)\n'

In [5]:
AP_LIST = meta['ap_list']
NUM_RECORDS = len(d)

In [6]:
df = pd.DataFrame(AP_LIST, columns = ['AP'])
temp = pd.DataFrame(d.values()).transpose()
temp.columns = d.keys()
df = pd.concat([df, temp] , axis = 1)
df = df.set_index('AP')

In [7]:
# dd = pd.DataFrame(AP_LIST, columns = ['AP'])
# for k in list(d.keys()):
#     temp = pd.DataFrame(d[k], columns=[k])
#     dd = pd.concat([dd, temp] , axis = 1)
# dd = dd.set_index('AP')

In [8]:
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [9]:
RP_LIST = sorted(list(set([x[:find_nth(x, '.', 2)] for x in list(df.columns)])))
RP_MAP = {key : i for i, key in enumerate(RP_LIST)}
TIMESTEPS = 5

In [10]:
radio_map = {}

for direction in ['north', 'south', 'east', 'west']:
    arr = np.ndarray((len(AP_LIST), len(RP_MAP), TIMESTEPS), dtype = np.float16)
    for key in RP_MAP:
        cols = [x for x in df.columns if key + '.' + direction in x]
        temp = df[cols].to_numpy()
        arr[:,RP_MAP[key],:] = temp 

    radio_map[direction] = arr


In [11]:
radio_map['north'][:,RP_MAP['B7.3'],0]

array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan, -82.,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan, -89.,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan, -73., -88.,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  n

In [12]:
radio_map.keys()

dict_keys(['north', 'south', 'east', 'west'])